In [1]:
from bokeh.plotting import figure, save, show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColumnDataSource, Range1d, HoverTool, LogColorMapper, BoxZoomTool
from bokeh.resources import CDN
from bokeh.palettes import Blues6 as palette
#from bokeh.tile_providers import CARTODBPOSITRON_RETINA
import pysal as ps
import geopandas as gpd
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
df = pd.read_excel("14.xlsx", encoding='utf-8')
PATH = r"mapdata/TOWN_MOI_1071031.shp"
gdf = gpd.read_file(PATH, encoding='utf-8') #全台灣村里界圖
gdf = gdf.to_crs({'init' :'epsg:4326'})
palette.reverse()

In [3]:
gdf.head()

,TOWNID,TOWNCODE,COUNTYNAME,TOWNNAME,TOWNENG,COUNTYID,COUNTYCODE,geometry
0,V02,10014020,臺東縣,成功鎮,Chenggong Township,V,10014,"POLYGON ((121.409815737 23.21369278500009, 121..."
1,T21,10013210,屏東縣,佳冬鄉,Jiadong Township,T,10013,"POLYGON ((120.5484580210001 22.46067220300006,..."
2,P13,10009130,雲林縣,麥寮鄉,Mailiao Township,P,10009,"POLYGON ((120.3019750420001 23.81624614400005,..."
3,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"(POLYGON ((121.491547243 22.67745703900005, 12..."
4,V16,10014160,臺東縣,蘭嶼鄉,Lanyu Township,V,10014,(POLYGON ((121.6117956420001 21.94289657000007...


In [4]:
ndf = pd.merge(gdf, df)
villages_shp = gpd.GeoDataFrame(ndf)

In [5]:
def cartesian(x): 
    return np.vstack(np.array([np.array(np.meshgrid(*i)).T.reshape(-1,9) for i in x.values]))

In [6]:
villages_shp = gpd.GeoDataFrame(cartesian(villages_shp),columns=villages_shp.columns)

In [7]:
villages_shp.head(6)

,TOWNID,TOWNCODE,COUNTYNAME,TOWNNAME,TOWNENG,COUNTYID,COUNTYCODE,geometry,RATE
0,V02,10014020,臺東縣,成功鎮,Chenggong Township,V,10014,"POLYGON ((121.409815737 23.21369278500009, 121...",9.57
1,T21,10013210,屏東縣,佳冬鄉,Jiadong Township,T,10013,"POLYGON ((120.5484580210001 22.46067220300006,...",14.19
2,P13,10009130,雲林縣,麥寮鄉,Mailiao Township,P,10009,"POLYGON ((120.3019750420001 23.81624614400005,...",13.18
3,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.491547243 22.67745703900005, 121...",13.44
4,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.512131544 22.68070962500008, 121...",13.44
5,V16,10014160,臺東縣,蘭嶼鄉,Lanyu Township,V,10014,"POLYGON ((121.6117956420001 21.94289657000007,...",9.97


In [8]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [9]:
villages_shp['x'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
villages_shp['y'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [10]:
villages_shp.head(6)

,TOWNID,TOWNCODE,COUNTYNAME,TOWNNAME,TOWNENG,COUNTYID,COUNTYCODE,geometry,RATE,x,y
0,V02,10014020,臺東縣,成功鎮,Chenggong Township,V,10014,"POLYGON ((121.409815737 23.21369278500009, 121...",9.57,"[121.40981573700003, 121.40984267700004, 121.4...","[23.21369278500009, 23.213661019000085, 23.213..."
1,T21,10013210,屏東縣,佳冬鄉,Jiadong Township,T,10013,"POLYGON ((120.5484580210001 22.46067220300006,...",14.19,"[120.5484580210001, 120.54853036000009, 120.54...","[22.46067220300006, 22.460667926000042, 22.460..."
2,P13,10009130,雲林縣,麥寮鄉,Mailiao Township,P,10009,"POLYGON ((120.3019750420001 23.81624614400005,...",13.18,"[120.30197504200008, 120.30197375900002, 120.3...","[23.816246144000047, 23.815993651000042, 23.81..."
3,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.491547243 22.67745703900005, 121...",13.44,"[121.49154724300001, 121.49184721100006, 121.4...","[22.677457039000046, 22.676760913000066, 22.67..."
4,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.512131544 22.68070962500008, 121...",13.44,"[121.51213154399998, 121.51203143500005, 121.5...","[22.680709625000077, 22.680408062000083, 22.68..."
5,V16,10014160,臺東縣,蘭嶼鄉,Lanyu Township,V,10014,"POLYGON ((121.6117956420001 21.94289657000007,...",9.97,"[121.61179564200006, 121.61133260100007, 121.6...","[21.942896570000073, 21.942768266000087, 21.94..."


In [11]:
town_xs = villages_shp.x
town_ys = villages_shp.y
town_name = villages_shp.COUNTYNAME + ", " +  villages_shp.TOWNNAME
town_rate = villages_shp.RATE

In [12]:
#villages_shp_source = GeoJSONDataSource(geojson=villages_shp.to_json())
source = ColumnDataSource(data=dict(
    x=town_xs,
    y=town_ys,
    cty=town_name,
    rate=town_rate
))

TOOLTIPS = [
    ("Latitude", "$x"),
    ("Longitude", "$y"),
    ("County", "@cty"), 
    ("Rate", "@rate{%0.2f}%")
]
color_mapper = LogColorMapper(palette=palette)# Color map
#HoverTool.mode="vline"

In [13]:
# Initialize our plot figure
p = figure(title="Taiwan Map", plot_width=800, plot_height=600,
          tools="pan,wheel_zoom,box_zoom,reset")
p.axis.visible=False
p.add_tools(HoverTool(
    tooltips=TOOLTIPS,
    
    formatters={
        'rate' : 'printf',   # use 'printf' formatter for 'rate' field
                             # use default 'numeral' formatter for other fields
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode='vline'
))

p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

zoom_overlay = p.select_one(BoxZoomTool).overlay

zoom_overlay.line_color = "olive"
zoom_overlay.line_width = 5
zoom_overlay.line_dash = "solid"
zoom_overlay.fill_color = None

In [14]:
# Add the lines to the map from our GeoJSONDataSource -object
#p.multi_line('x', 'y', source=source, color='black', line_width=2)
p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.3)

GlyphRenderer(id='1048', ...)

In [15]:
output_file("taiwanmap.html")
#output_notebook()

In [16]:
#show(p)
save(p, title="taiwanmap")

'C:\\Users\\User\\Documents\\Jupyter\\taiwanmap.html'